# Support Vector Machines (SVM)

Write a class `MySVM` for a binary classification task and compare the prediction with `sklearn` model.

In [6]:
#!pip install cvxopt

In [7]:
import numpy as np
import pandas as pd
from cvxopt import matrix, solvers
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from sklearn.svm import SVC
from sklearn.datasets import load_wine
from ucimlrepo import fetch_ucirepo

In [8]:
# # Load classification dataset https://archive.ics.uci.edu/dataset/17/breast+cancer+wisconsin+diagnostic
# dataset = fetch_ucirepo(id=17)
# X = dataset.data.features
# y = dataset.data.targets
# y = y.values.ravel() # flatten to 1D array
# y = (y == 'M').astype(int)  # Convert labels to 0 and 1

In [9]:
X = load_wine(return_X_y=True)[0][:200, :4]
y = load_wine(return_X_y=True)[1][:200]
y = np.where(y == 0, -1, 1)  # Convert to -1 and 1 labels

X.shape, y.shape

((178, 4), (178,))

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((142, 4), (142,), (36, 4), (36,))

In [11]:
def linear_kernel(X, Y):
    """Linear kernel: K(x_i, x_j) = x_i^T x_j"""
    return X @ Y.T


def rbf_kernel(X, Y, gamma=1.0):
    """RBF kernel."""
    X_norm = np.sum(X**2, axis=1).reshape(-1, 1)
    Y_norm = np.sum(Y**2, axis=1).reshape(1, -1)
    sq_dists = X_norm + Y_norm - 2 * (X @ Y.T)
    return np.exp(-gamma * sq_dists)


def polynomial_kernel(X, Y, degree=3, coef0=1.0):
    """Polynomial kernel."""
    return (X @ Y.T + coef0) ** degree


class MySVM:
    """
    C-SVM solved via QP (cvxopt).

    Dual problem:
        max  1^T α - 1/2 α^T (Y K Y) α
        s.t. 0 ≤ α_i ≤ C,   ∑ α_i y_i = 0

    If C is None, use hard-margin (only α_i ≥ 0).
    """

    def __init__(self, kernel=linear_kernel, C=1.0):
        self.kernel = kernel
        self.C = C

        self.alpha = None
        self.b = 0.0
        self.support_idx = None
        self.X_train = None
        self.y_train = None
        self.w = None

    def fit(self, X, y):
        X = np.asarray(X, dtype=float)
        y = np.asarray(y, dtype=float)

        # Ensure labels are -1 and +1
        unique = np.unique(y)
        if set(unique) == {0, 1}:
            y = np.where(y == 0, -1.0, 1.0)
        elif set(unique) == {-1, 1}:
            y = y.astype(float)
        else:
            raise ValueError("Labels must be binary {0,1} or {-1,1}.")

        self.X_train = X
        self.y_train = y
        n_samples = X.shape[0]

        # Gram matrix
        K = self.kernel(X, X)

        # Build matrices for cvxopt QP:
        #  minimize  (1/2) x^T P x + q^T x
        #  s.t.      G x <= h
        #            A x = b
        #
        # Here, x = α.
        # P = Y K Y, q = -1
        Y = np.diag(y)
        P = matrix(Y @ K @ Y, tc='d')
        q = matrix(-np.ones(n_samples), tc='d')

        # Equality constraint: y^T α = 0
        A = matrix(y.reshape(1, -1), tc='d')
        b = matrix(0.0, tc='d')

        # Inequality constraints:
        # hard margin: α >= 0  ->  -I α <= 0
        # soft margin: 0 <= α <= C ->
        #   [ I] α <= [ C]
        #   [-I]     [ 0]
        if self.C is None:
            # Only α >= 0
            G = matrix(-np.eye(n_samples), tc='d')
            h = matrix(np.zeros(n_samples), tc='d')
        else:
            G_top = np.eye(n_samples)
            G_bottom = -np.eye(n_samples)
            G = np.vstack([G_top, G_bottom])

            h_top = np.full(n_samples, self.C)
            h_bottom = np.zeros(n_samples)
            h = np.hstack([h_top, h_bottom])

            G = matrix(G, tc='d')
            h = matrix(h, tc='d')

        # Turn off cvxopt output if you want it quiet:
        solvers.options['show_progress'] = False

        solution = solvers.qp(P, q, G, h, A, b)
        alpha = np.array(solution['x']).reshape(-1)

        self.alpha = alpha

        # Support vectors: α_i > 0
        sv_mask = alpha > 1e-6
        self.support_idx = np.where(sv_mask)[0]

        # Compute bias b:
        # For any SV i with 0 < α_i < C:
        #   y_i (w^T φ(x_i) + b) = 1
        # => b = y_i - sum_j α_j y_j K_ji
        if self.support_idx.size == 0:
            self.b = 0.0
        else:
            # For numerical stability, use only "free" SVs
            if self.C is None:
                free_sv = self.support_idx
            else:
                free_sv = [
                    i for i in self.support_idx
                    if 1e-6 < alpha[i] < self.C - 1e-6
                ]
                if not free_sv:
                    # Fallback: use all SVs
                    free_sv = list(self.support_idx)

            b_list = []
            for i in free_sv:
                decision_no_b = np.sum(
                    alpha * y * K[:, i]
                )
                b_list.append(y[i] - decision_no_b)
            self.b = float(np.mean(b_list))

        # Recover w for linear kernel
        if self.kernel is linear_kernel:
            self.w = (alpha * y) @ X
        else:
            self.w = None

    def decision_function(self, X):
        X = np.asarray(X, dtype=float)
        if self.w is not None:
            return X @ self.w + self.b

        K_test = self.kernel(X, self.X_train)
        return (K_test * (self.alpha * self.y_train)).sum(axis=1) + self.b

    def predict(self, X):
        scores = self.decision_function(X)
        return np.where(scores >= 0, 1, -1)

In [12]:
svm = MySVM(kernel=linear_kernel, C=1e10)
svm.fit(X_train, y_train)
y_pred = svm.predict(X_test)

print("Support vectors:", svm.support_idx)
print("w:", svm.w)
print("b:", svm.b)

accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred)

print(f"My model: ACC={accuracy:.4f}, F1={f1:.4f}, ROC AUC={roc_auc:.4f}")

Support vectors: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141]
w: [-2.14910889  0.30120087 -6.50523376  0.88122559]
b: 25.744209397007044
My model: ACC=0.8889, F1=0.9048, ROC AUC=0.8961


In [13]:
sk_model = SVC(kernel='linear', C=1e10)
sk_model.fit(X_train, y_train)
sk_y_pred = sk_model.predict(X_test)

print("SK Learn Support vectors:", sk_model.support_)
print("SK Learn w:", sk_model.coef_)
print("SK Learn b:", sk_model.intercept_)

sk_accuracy = accuracy_score(y_test, sk_y_pred)
sk_f1 = f1_score(y_test, sk_y_pred)
sk_roc_auc = roc_auc_score(y_test, sk_y_pred)

print(f"My model: ACC={accuracy:.4f}, F1={f1:.4f}, ROC AUC={roc_auc:.4f}")
print(f"SK Learn: ACC={sk_accuracy:.4f}, F1={sk_f1:.4f}, ROC AUC={sk_roc_auc:.4f}")

SK Learn Support vectors: [ 30  32  44  49  81  82  89   7  54  73  95 119 127 135]
SK Learn w: [[-1625003.29431152   588990.77801132 -6482709.32610655   609742.50299072]]
SK Learn b: [24681022.8087939]
My model: ACC=0.8889, F1=0.9048, ROC AUC=0.8961
SK Learn: ACC=0.9167, F1=0.9333, ROC AUC=0.9058
